***GENERATED CODE FOR prophet3 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
import pyspark
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = transformationDF.drop('_c0')
        transformationDF = transformationDF.drop('AdmitDate')
        transformationDF = transformationDF.drop('gender')
        transformationDF = transformationDF.drop('facid')
        transformationDF = transformationDF.drop('New date28')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverManualProphet(df, timeColumn, tocompare):
    df.rename(columns={timeColumn: "ds", tocompare: "y"}, inplace=True)
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run prophet3Hooks.ipynb
try:
	#sourcePreExecutionHook()

	monthtetneeu = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-198e8790-fcb2-4bb1-abd4-57a5d694a2ab-c000.csv', 'filename': '171678721312monthstest.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/TimeSeriesTesting/12monthstest.csv', 'viewFileName': '12monthstest.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run prophet3Hooks.ipynb
try:
	#transformationPreExecutionHook()

	prophetfeatureforecast = TransformationTimeSeriesForecastingMain.run(monthtetneeu, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{"feature_label": "_c0", "threshold": 24439.508196721312, "transformation_label": "Binarizer"}], "feature": "_c0", "type": "real", "selected": "False", "replaceby": "mean", "stats": {"count": "18", "mean": "2222.28", "stddev": "1402.67", "min": "0.0", "max": "4446.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "_c0"}, {"transformationsData": [{"feature_label": "AdmitDate", "transformation_label": "String Indexer"}], "feature": "AdmitDate", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "18", "mean": "", "stddev": "", "min": "01/01/2012", "max": "01/18/2012", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "1.0", "stddev": "0.0", "min": "1.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "lengthofstay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rcount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rcount"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "18", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dialysisrenalendstage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dialysisrenalendstage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asthma", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "asthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "irondef", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "irondef"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pneum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pneum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "substancedependence", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "substancedependence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychologicaldisordermajor", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "psychologicaldisordermajo..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depress", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "depress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychother", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "psychother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fibrosisandother", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fibrosisandother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "malnutrition", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "malnutrition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hemo", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hemo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "11.67", "stddev": "1.24", "min": "10.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "9.94", "stddev": "1.86", "min": "9.0", "max": "16.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "137.44", "stddev": "2.87", "min": "130.0", "max": "142.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "141.83", "stddev": "32.23", "min": "86.0", "max": "202.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "12.0", "stddev": "0.69", "min": "10.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "0.67", "stddev": "0.49", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "29.33", "stddev": "1.97", "min": "25.0", "max": "33.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "74.0", "stddev": "11.0", "min": "59.0", "max": "106.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "6.0", "stddev": "0.0", "min": "6.0", "max": "6.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "secondarydiagnosisnonicd9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18", "mean": "3.0", "stddev": "3.45", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "secondarydiagnosisnonicd9"}, {"transformationsData": [{"feature_label": "facid", "transformation_label": "String Indexer"}], "feature": "facid", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "18", "mean": "", "stddev": "", "min": "A", "max": "B", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "facid"}, {"transformationsData": [{}], "feature": "New Date27", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "New Date27"}, {"transformationsData": [{}], "feature": "New date28", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "New date28"}], "features": {"timecolumn": "New Date27", "tocompare": "lengthofstay"}, "dataPercentage": "100", "originalfile": "/FileStore/platform/uploadedSourceFiles/part-00000-198e8790-fcb2-4bb1-abd4-57a5d694a2ab-c000.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(prophetfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run prophet3Hooks.ipynb
try:
	#mlPreExecutionHook()

	modelData = driverManualProphet(prophetfeatureforecast, 'New Date27', 'lengthofstay')

	#mlPostExecutionHook(modelData)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
